In [ ]:
!pip install selenium

In [ ]:
!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -fy install

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:7 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:12 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,222 B]
Fetched 3,047 B in 3s (1,218 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repos

In [ ]:
!wget https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/131.0.6778.85/linux64/chromedriver-linux64.zip
!unzip -o chromedriver-linux64.zip
!mv chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver

--2024-12-16 08:13:35--  https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/131.0.6778.85/linux64/chromedriver-linux64.zip
Resolving edgedl.me.gvt1.com (edgedl.me.gvt1.com)... 34.104.35.123, 2600:1900:4110:86f::
Connecting to edgedl.me.gvt1.com (edgedl.me.gvt1.com)|34.104.35.123|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/chrome-for-testing-public/131.0.6778.85/linux64/chromedriver-linux64.zip [following]
--2024-12-16 08:13:35--  https://storage.googleapis.com/chrome-for-testing-public/131.0.6778.85/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.207, 108.177.127.207, 172.217.218.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9925053 (9.5M) [application/zip]
Saving to: ‘chromedriver-linux64.zip.1’

chromedriver-linux6 100%[===================>

In [ ]:
!google-chrome --version
!chromedriver --version

Google Chrome 131.0.6778.139 
ChromeDriver 131.0.6778.85 (3d81e41b6f3ac8bcae63b32e8145c9eb0cd60a2d-refs/branch-heads/6778@{#2285})


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import openpyxl

# Chrome 옵션 설정
options = Options()
options.add_argument('--headless')  # 브라우저 창 없이 실행
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# ChromeDriver 서비스 설정
service = Service('/usr/bin/chromedriver')

# WebDriver 생성
driver = webdriver.Chrome(service=service, options=options)

# 엑셀 파일 생성
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Medicine Info"

# 엑셀 파일 헤더 추가
ws.append(["약 이름", "약 ATC 코드", "약 정보", "약 주의 사항"])

# URL의 번호 범위 설정
start_index = 6516
end_index = 6520

for i in range(start_index, end_index + 1):
    url = f'https://www.medicines.org.uk/emc/product/{i}/smpc'
    driver.get(url)

    try:
        # 페이지 로딩 대기 시간
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        # 쿠키 배너 닫기 시도 시간
        try:
            cookie_banner = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
            )
            cookie_banner.click()
        except:
            pass

        # "Expand All" 버튼 클릭 시간
        try:
            WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "expand-all-btn"))
            ).click()
        except:
            pass

        # 주요 정보 추출 대기시간
        medicine_name = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#smpc > div > details:nth-child(1) > div > p"))
        ).text

        medicine_ATC = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#about-medicine > div:nth-child(2)"))
        ).text

        medicine_info = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#smpc > div > details:nth-child(4) > details:nth-child(3) > div > p"))
        ).text

        warning_1 = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#smpc > div > details:nth-child(4) > details:nth-child(6) > div"))
        ).text

        # 결과 출력
        print("약 이름:", medicine_name)
        print("약 ATC 코드:", medicine_ATC)
        print("약 정보:", medicine_info)
        print("약 주의 사항:", warning_1)
        print("\n\n")

        # 데이터를 엑셀에 추가
        ws.append([medicine_name, medicine_ATC, medicine_info, warning_1])

    except Exception as e:
        print(f"Error for {url}: {e}")
        continue

# 드라이버 종료
driver.quit()

# 엑셀 파일 저장
output_file = "PillGood_Medicine_Info.xlsx"
wb.save(output_file)
print(f"Data saved to {output_file}")


약 이름: Stalevo 50 mg/12.5 mg/200 mg film-coated tablets
약 ATC 코드: N04BA03
약 정보: Stalevo is indicated for the treatment of adult patients with Parkinson's disease and end-of-dose motor fluctuations not stabilised on levodopa/dopa decarboxylase (DDC) inhibitor treatment.
약 주의 사항: - Stalevo is not recommended for the treatment of drug-induced extrapyramidal reactions.
- Stalevo therapy should be administered cautiously to patients with ischemic heart disease, severe cardiovascular or pulmonary disease, bronchial asthma, renal or endocrine disease, history of peptic ulcer disease or history of convulsions.
- In patients with a history of myocardial infarction who have residual atrial nodal or ventricular arrhythmias; cardiac function should be monitored with particular care during the period of initial dose adjustments.
- All patients treated with Stalevo should be monitored carefully for the development of mental changes, depression with suicidal tendencies, and other serious antisocial be